In [1]:
import wandb
from wandb.keras import WandbMetricsLogger


import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM,Conv1D,MaxPool1D,Bidirectional,Concatenate
from tensorflow.keras.models import Model

from TranscriptionDataset.TensorflowTuningSequence import TuningSequence
from TUtils import get_timestamp

import numpy as np

print("TensorFlow version:", tf.__version__)

2023-06-22 21:55:28.060314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-22 21:55:28.683697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0


In [ ]:
logdir = f"TBLogs/{get_timestamp()}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
wandb.tensorboard.patch(root_logdir=logdir)
wandb.init(project='TFArrangement')
print(f"Tensorboard : {logdir}")

In [ ]:
trainSequence = TuningSequence("Trainsets/S_Tier_Mel.hdf5")
testSequence = TuningSequence("Trainsets/Mini_Validation_Test.hdf5",shuffle=False)
# trainSequence[0][0].shape,trainSequence[0][1].shape

In [4]:
input_layer = Input(shape=(None,256),batch_size=1)
arr_input = Input(shape=6)

x = (Conv1D(data_format="channels_last",filters=128, kernel_size=5, padding="same", activation="relu"))(input_layer)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=64, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=8, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = Dense(128,activation="relu")(x)
x = Bidirectional(LSTM(128))(x)
x = Dense(128,activation="relu")(x)
y = Concatenate()([x,arr_input])
y = Dense(64,activation="relu")(y)
y = Dense(32,activation="relu")(y)
# output = Dense(1,activation="sigmoid")(x)
outputs = []
for i in range(6):
    outputs.append(Dense(1,name=f'tuning_{i}',activation="sigmoid")(y))

model = Model(inputs=input_layer, outputs=outputs)

print(model.summary())

2023-06-22 22:00:21.482794: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-22 22:00:21.484314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-22 22:00:21.485336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 6), dtype=tf.float32, name='input_6'), name='input_6', description="created by layer 'input_6'") at layer "concatenate_2". The following previous layers were accessed without issue: ['conv1d_6', 'max_pooling1d_6', 'conv1d_7', 'max_pooling1d_7', 'conv1d_8', 'max_pooling1d_8', 'dense_8', 'bidirectional_2', 'dense_9']